# MusicGen-Large Finetuning Loop (Sequential Processing) - Transformers Version

このノートブックは、大規模なデータセットをZIPファイル単位で順次処理（解凍→学習→削除）しながらMusicGen-Largeをファインチューニングします。
Hugging Face Transformersライブラリを使用します。

## 前提条件
1. Google Driveに以下のデータがあること
   - `MyData/Archive_wavs/metadata.jsonl`: 全データのメタデータ
   - `MyData/Archive_wavs/archive_batch_xxxx.zip`: 音声データのZIPファイル群
2. A100 GPU推奨（VRAM容量のため）
3. **WandB API Key**: Colabのシークレット（鍵マーク）に `WANDB_API_KEY` という名前で登録してください。

In [1]:
import os
import subprocess
import sys

print("Uninstalling potentially problematic libraries...")
# 関連するライブラリを一度アンインストール
!pip uninstall -y torch torchvision torchaudio torchcodec
print("Uninstall complete.")

print("Installing libraries...")

# CUDA 12.6対応のPyTorch (Nightly or Pre-release)
# 注意: ユーザー指定によりCUDA 12.6をターゲットにします。
!pip install --pre torch torchvision torchaudio torchcodec --index-url https://download.pytorch.org/whl/nightly/cu126

# Hugging Face Libraries & WandB
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U datasets accelerate bitsandbytes wandb

# FFmpegのインストール
!apt-get update
!apt-get install -y ffmpeg
print("FFmpeg installation complete.")

print("Installation complete.")

Uninstalling potentially problematic libraries...
Found existing installation: torch 2.10.0.dev20251203+cu126
Uninstalling torch-2.10.0.dev20251203+cu126:
  Successfully uninstalled torch-2.10.0.dev20251203+cu126
Found existing installation: torchvision 0.25.0.dev20251204+cu126
Uninstalling torchvision-0.25.0.dev20251204+cu126:
  Successfully uninstalled torchvision-0.25.0.dev20251204+cu126
Found existing installation: torchaudio 2.10.0.dev20251204+cu126
Uninstalling torchaudio-2.10.0.dev20251204+cu126:
  Successfully uninstalled torchaudio-2.10.0.dev20251204+cu126
Found existing installation: torchcodec 0.9.0.dev20251204+cu126
Uninstalling torchcodec-0.9.0.dev20251204+cu126:
  Successfully uninstalled torchcodec-0.9.0.dev20251204+cu126
Uninstall complete.
Installing libraries...
Looking in indexes: https://download.pytorch.org/whl/nightly/cu126
  Using cached https://download.pytorch.org/whl/nightly/cu126/torch-2.10.0.dev20251204%2Bcu126-cp312-cp312-manylinux_2_28_x86_64.whl.metadata 

In [2]:
# @title 1.5 WandB ログイン
import wandb
from google.colab import userdata

try:
    wandb_api_key = userdata.get('WANDB_API_KEY')
    wandb.login(key=wandb_api_key)
    print("Logged in to WandB successfully.")
except Exception as e:
    print(f"WandB login failed: {e}")
    print("Please ensure 'WANDB_API_KEY' is set in Colab secrets.")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: charge0315 (charge0315-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Logged in to WandB successfully.


In [3]:
# @title 2. Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# @title 3. パスと設定の定義
import os
from pathlib import Path

# --- ユーザー設定エリア ---
DRIVE_ROOT = Path('/content/drive/MyDrive')
DATA_ROOT = DRIVE_ROOT / 'Archive_Wavs'
METADATA_PATH = DATA_ROOT / 'metadata.jsonl'
ZIP_DIR = DATA_ROOT

# 出力先（チェックポイント保存場所）
OUTPUT_DIR = DRIVE_ROOT / 'MusicGen_Finetuning_Output'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# 一時作業ディレクトリ（Colabローカル）
TEMP_WORK_DIR = Path('/content/temp_work')
TEMP_DATA_DIR = TEMP_WORK_DIR / 'data'

TEMP_DATA_DIR.mkdir(exist_ok=True, parents=True)

print(f"Metadata: {METADATA_PATH}")
print(f"Zip Dir: {ZIP_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

Metadata: /content/drive/MyDrive/Archive_Wavs/metadata.jsonl
Zip Dir: /content/drive/MyDrive/Archive_Wavs
Output Dir: /content/drive/MyDrive/MusicGen_Finetuning_Output


In [5]:
import os

# DATA_ROOT はすでに定義されているはずです。
# 定義されていない場合は、セル a1f847a0 を実行してください。

print(f"Listing contents of {DATA_ROOT}:")
files_in_dir = os.listdir(DATA_ROOT)
if files_in_dir:
    for f in files_in_dir:
        print(f)
else:
    print("Directory is empty or does not exist.")

Listing contents of /content/drive/MyDrive/Archive_Wavs:
train.jsonl
valid.jsonl
archive_batch_0001.zip
archive_batch_0002.zip
archive_batch_0003.zip
archive_batch_0004.zip
archive_batch_0005.zip
archive_batch_0006.zip
archive_batch_0007.zip
archive_batch_0008.zip
archive_batch_0009.zip
archive_batch_0010.zip
archive_batch_0011.zip
archive_batch_0012.zip
archive_batch_0013.zip
archive_batch_0014.zip
archive_batch_0015.zip
archive_batch_0016.zip
archive_batch_0017.zip
archive_batch_0018.zip
archive_batch_0019.zip
archive_batch_0020.zip
archive_batch_0021.zip
processed_files.txt
metadata.jsonl
archive_batch_0022.zip
archive_batch_0023.zip
archive_batch_0024.zip
archive_batch_0025.zip
archive_batch_0026.zip
archive_batch_0027.zip
archive_batch_0028.zip
archive_batch_0029.zip
archive_batch_0030.zip
archive_batch_0031.zip
archive_batch_0032.zip
archive_batch_0033.zip
archive_batch_0034.zip
archive_batch_0035.zip
archive_batch_0036.zip
archive_batch_0037.zip
archive_batch_0038.zip
archive_ba

In [46]:
import json
import shutil
import subprocess
import glob
import torchaudio
import numpy as np # Add numpy import
from datasets import load_dataset, Audio

def extract_zip(zip_path, extract_to):
    """ZIPファイルを指定ディレクトリに解凍する"""
    print(f"Extracting {zip_path} to {extract_to}...")
    if extract_to.exists():
        shutil.rmtree(extract_to)
    extract_to.mkdir(parents=True, exist_ok=True)

    subprocess.run(['unzip', '-q', str(zip_path), '-d', str(extract_to)], check=True)
    print("Extraction complete.")

def create_batch_metadata(main_metadata_path, current_wav_dir, output_jsonl_path):
    """
    メインのmetadata.jsonlから、現在解凍されているファイルに対応するエントリのみを抽出し、
    パスをColab上の絶対パスに書き換えて新しいjsonlを作成する。
    """
    print(f"Creating batch metadata at {output_jsonl_path}...")

    extracted_files = list(current_wav_dir.rglob('*.wav'))
    extracted_files_map = {f.name: f for f in extracted_files}

    valid_entries = []

    with open(main_metadata_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                orig_path = entry.get('path', '')
                filename = os.path.basename(orig_path)

                if filename in extracted_files_map:
                    # パスを絶対パスに更新
                    entry['path'] = str(extracted_files_map[filename])
                    # TransformersのDatasetで読み込むために 'audio' キーにパスを入れるのが一般的だが
                    # ここでは後処理でロードするため 'path' のままでもOK。
                    # ただし、datasets libraryのAudio機能を使うなら 'audio': path が便利。
                    entry['audio'] = str(extracted_files_map[filename])
                    valid_entries.append(entry)
            except json.JSONDecodeError:
                continue

    if not valid_entries:
        print("Warning: No matching metadata found for extracted files.")
        return False

    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')

    print(f"Created metadata with {len(valid_entries)} entries.")
    return True

def preprocess_function(examples, processor, model, audio_column_name="audio", text_column_name="caption"):
    """データセットの前処理関数"""
    processed_audio_arrays = []
    valid_indices = []
    # Initialize sampling_rate, assuming all samples in a batch have the same rate.
    # It's safer to get it from the first valid sample.
    sampling_rate = None

    # Filter and process audio arrays
    for idx, audio_info in enumerate(examples[audio_column_name]):
        audio_array = audio_info["array"]
        if audio_array is not None and len(audio_array) > 0:
            if sampling_rate is None:
                sampling_rate = audio_info["sampling_rate"]

            # Convert 2D (stereo) arrays to 1D (mono) by averaging channels if necessary
            if audio_array.ndim == 2:
                # Assuming format is (channels, samples) or (samples, channels)
                # Take mean across the channel axis to convert to mono
                # Check which axis has the smaller dimension (likely channels)
                if audio_array.shape[0] < audio_array.shape[1]: # (channels, samples)
                    audio_array = np.mean(audio_array, axis=0)
                else: # (samples, channels)
                    audio_array = np.mean(audio_array, axis=1)

            # Ensure it's a numpy array of float32 and 1D
            if audio_array.ndim == 1:
                processed_audio_arrays.append(audio_array.astype(np.float32))
                valid_indices.append(idx)
            else:
                print(f"Skipping problematic audio sample at index {idx} in batch: Not 1D after conversion attempt.")

        else:
            print(f"Skipping problematic audio sample at index {idx} in batch: Empty or None.")

    if not processed_audio_arrays:
        # If all audio samples in the batch were problematic, return empty inputs
        # This will be filtered by the dataset.map later by default if remove_columns=False is not set
        # or lead to an empty batch for the model.
        return {}

    # Filter texts based on valid_indices
    texts = [examples.get(text_column_name, [""] * len(examples[audio_column_name]))[i] for i in valid_indices]
    # Handle other text key if primary is empty or missing
    for i, text_val in enumerate(texts):
        if not text_val:
            if "text" in examples and valid_indices[i] < len(examples["text"]):
                texts[i] = examples["text"][valid_indices[i]]
            elif "description" in examples and valid_indices[i] < len(examples["description"]):
                texts[i] = examples["description"][valid_indices[i]]
    # Ensure all texts are strings
    texts = [str(t) if t else "" for t in texts]


    # Define max lengths for audio and text
    # Musicgen typically processes audio up to ~30 seconds (32kHz * 30s = 960,000 samples)
    # Using a common max_length for Encodec (e.68 seconds = 245760 samples)
    MAX_AUDIO_SAMPLES = 245760 # Approx 7.68 seconds at 32kHz
    MAX_TEXT_TOKENS = 256

    # Process audio inputs using the feature extractor
    # Do not return_tensors="pt" here, get raw numpy arrays (or lists of arrays) first
    audio_features = processor.feature_extractor(
        processed_audio_arrays, # Use cleaned audio arrays
        sampling_rate=sampling_rate,
        padding="max_length",
        max_length=MAX_AUDIO_SAMPLES,
        # truncation=True, # No longer needed, padding='max_length' handles this for feature_extractor
        # return_tensors="pt", # REMOVED: we will convert manually
    )

    # Manually ensure consistent numpy array for input_values and attention_mask before converting to tensor
    # audio_features.input_values might be a list of np.arrays, if so, stack them
    # Ensure they are numerical dtype, not object

    # Ensure audio_features.input_values is always a list for robust iteration
    input_values_to_normalize = audio_features.input_values
    if not isinstance(input_values_to_normalize, list):
        input_values_to_normalize = [input_values_to_normalize]

    # Ensure all audio inputs are exactly MAX_AUDIO_SAMPLES long BEFORE stacking
    normalized_input_values = []
    for audio_arr in input_values_to_normalize:
        # Robustly handle potentially malformed audio_arr from feature_extractor
        if not isinstance(audio_arr, np.ndarray):
            print(f"Skipping malformed audio_arr in input_values: type={type(audio_arr)}")
            continue

        # Force to 1D if it's 2D (e.g., from feature_extractor itself producing 2D output)
        if audio_arr.ndim == 2:
            # Assuming format is (channels, samples) or (samples, channels)
            # Take mean across the channel axis to convert to mono
            if audio_arr.shape[0] < audio_arr.shape[1]: # (channels, samples)
                audio_arr = np.mean(audio_arr, axis=0)
            else: # (samples, channels)
                audio_arr = np.mean(audio_arr, axis=1)

        if audio_arr.ndim != 1:
            print(f"Skipping problematic audio sample from feature_extractor: Not 1D after conversion attempt (ndim={audio_arr.ndim}).")
            continue

        current_length = audio_arr.shape[-1]
        if current_length > MAX_AUDIO_SAMPLES:
            normalized_input_values.append(audio_arr[:MAX_AUDIO_SAMPLES])
        elif current_length < MAX_AUDIO_SAMPLES:
            normalized_input_values.append(np.pad(audio_arr, (0, MAX_AUDIO_SAMPLES - current_length), 'constant', constant_values=0))
        else:
            normalized_input_values.append(audio_arr)

    if not normalized_input_values: # If all were skipped or originally empty, return empty
        return {}

    input_values_np = np.stack(normalized_input_values).astype(np.float32)

    # Reshape to (batch_size, 1, sequence_length) for mono audio
    input_values_np = input_values_np[:, np.newaxis, :]

    # Convert to PyTorch tensor
    audio_inputs_tensor = {
        "input_values": torch.from_numpy(input_values_np),
    }

    # Handle attention_mask similarly if it exists or create it
    if "attention_mask" in audio_features and audio_features.attention_mask is not None:
        # Ensure audio_features.attention_mask is always a list for robust iteration
        attention_mask_to_normalize = audio_features.attention_mask
        if not isinstance(attention_mask_to_normalize, list):
            attention_mask_to_normalize = [attention_mask_to_normalize]

        normalized_attention_mask = []
        for mask_arr in attention_mask_to_normalize:
            # Robustly handle potentially malformed mask_arr
            if not isinstance(mask_arr, np.ndarray):
                print(f"Skipping malformed mask_arr in attention_mask: type={type(mask_arr)}")
                continue

            # Force to 1D if it's 2D
            if mask_arr.ndim == 2:
                # Assuming format is (channels, samples) or (samples, channels)
                if mask_arr.shape[0] < mask_arr.shape[1]: # (channels, samples)
                    mask_arr = np.mean(mask_arr, axis=0).astype(np.longlong) # Ensure type after mean
                else: # (samples, channels)
                    mask_arr = np.mean(mask_arr, axis=1).astype(np.longlong)

            if mask_arr.ndim != 1:
                print(f"Skipping problematic mask_arr from feature_extractor: Not 1D after conversion attempt (ndim={mask_arr.ndim}).")
                continue

            current_length = mask_arr.shape[-1]
            if current_length > MAX_AUDIO_SAMPLES:
                normalized_attention_mask.append(mask_arr[:MAX_AUDIO_SAMPLES])
            elif current_length < MAX_AUDIO_SAMPLES:
                normalized_attention_mask.append(np.pad(mask_arr, (0, MAX_AUDIO_SAMPLES - current_length), 'constant', constant_values=0))
            else:
                normalized_attention_mask.append(mask_arr)

        if not normalized_attention_mask: # If all were skipped or originally empty, fallback to default
            audio_inputs_tensor["attention_mask"] = torch.ones_like(audio_inputs_tensor["input_values"], dtype=torch.long)
        else:
            attention_mask_np = np.stack(normalized_attention_mask).astype(np.longlong)
            audio_inputs_tensor["attention_mask"] = torch.from_numpy(attention_mask_np)
    else:
        # Fallback if feature_extractor still doesn't provide attention_mask or it's problematic
        audio_inputs_tensor["attention_mask"] = torch.ones_like(audio_inputs_tensor["input_values"], dtype=torch.long)


    # Process text inputs using the tokenizer
    text_inputs = processor.tokenizer(
        text=texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_TEXT_TOKENS,
        return_tensors="pt",
    )

    # --- Explicitly generate audio codes for labels and decoder_input_ids ---
    # This part needs the model's audio_encoder, so 'model' is passed to preprocess_function
    with torch.no_grad(): # Ensure this part does not affect gradients if run inside map
        # Move inputs to model device for encoding if not already there
        input_values_for_encode = audio_inputs_tensor["input_values"].to(model.device).to(torch.float16) # Explicitly cast to float16
        padding_mask_for_encode = audio_inputs_tensor["attention_mask"].to(model.device)

        # Add a check for non-finite values before passing to encoder
        if not torch.isfinite(input_values_for_encode).all():
            print(f"Warning: input_values_for_encode contains non-finite values (NaN/Inf). Skipping batch.")
            return {}

        try:
            audio_codes, audio_scales, _ = model.audio_encoder.encode(
                input_values_for_encode,
                padding_mask_for_encode,
                model.audio_encoder.config.target_bandwidths[-1] # Use model's audio_encoder's highest target_bandwidth config
            )
        except Exception as e:
            print(f"ERROR: An exception occurred during model.audio_encoder.encode: {e}")
            return {} # Skip batch on encoder error

        # Check if audio_codes is a tensor after encode call
        if not isinstance(audio_codes, torch.Tensor):
            print(f"ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: {type(audio_codes)}")
            print(f"Returned value: {audio_codes}")
            return {}

        # Shift audio codes to the right for training (as expected by causal LM decoder)
        # labels are the target audio codes (unshifted)
        # decoder_input_ids are the input to the decoder (shifted right)
        # Ensure the last dimension is not smaller than 1 for slicing
        if audio_codes.shape[-1] > 1:
            labels_audio_codes = audio_codes[:, 1:]
            decoder_input_ids_audio_codes = audio_codes[:, :-1]
        else:
            # Handle case where audio_codes might be too short after encoding
            # This could result in empty labels, which Trainer won't like.
            # For now, we'll make them identical if too short to shift.
            print("Warning: Audio codes too short for shifting, setting labels and decoder_input_ids to be the same.")
            labels_audio_codes = audio_codes
            decoder_input_ids_audio_codes = audio_codes

        # Move labels and decoder_input_ids back to CPU if needed by dataset/trainer, or keep on device
        labels_audio_codes = labels_audio_codes.cpu()
        decoder_input_ids_audio_codes = decoder_input_ids_audio_codes.cpu()

    # Combine the processed inputs for the model
    # Musicgen expects audio as 'input_values' and 'padding_mask' for the audio encoder,
    # and 'input_ids' and 'attention_mask' for the text encoder.
    # For Trainer to work correctly, we explicitly provide 'decoder_input_ids' and 'labels' (audio codes).
    inputs = {
        "input_values": audio_inputs_tensor["input_values"],    # Raw audio for encoder (can be ignored by model if decoder_input_ids are present)
        "padding_mask": audio_inputs_tensor["attention_mask"],   # Audio attention mask
        "input_ids": text_inputs.input_ids,                      # Text input IDs for the text encoder
        "attention_mask": text_inputs.attention_mask,            # Text attention mask for the text encoder
        "decoder_input_ids": decoder_input_ids_audio_codes,      # Shifted audio codes for decoder input
        "labels": labels_audio_codes,                            # Target audio codes for loss computation
    }
    return inputs

In [47]:
# @title 5. メインループ実行
import torch
from transformers import AutoProcessor, MusicgenForConditionalGeneration, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, Audio
from peft import LoraConfig, get_peft_model # PEFTライブラリをインポート

# モデルとプロセッサの準備
MODEL_ID = "facebook/musicgen-large"
print(f"Loading model: {MODEL_ID}...")

# 8-bit 量子化設定
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16 # A100/V100ならfloat16を推奨
)

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = MusicgenForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config, # 8-bit 量子化を適用
    device_map="auto" # 自動的にデバイスにマッピング
)

# PEFT (LoRA) の設定
# MusicGenのエンコーダ (T5EncoderModel) とデコーダ (MusicgenForCausalLM) の両方にLoRAを適用
lora_config = LoraConfig(
    r=8,  # LoRAのランク
    lora_alpha=32, # LoRAスケーリング係数
    target_modules=["q_proj", "v_proj"], # LoRAを適用するモジュール (Attention層のQuery, Value)
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM", # MusicGenはSequence-to-Sequenceモデル
)

# PEFTモデルをオリジナルモデルにアタッチ
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # 学習可能なパラメータ数を確認

model.train()

# ZIPファイルリスト取得
zip_files = sorted(list(ZIP_DIR.glob('archive_batch_*.zip')))
print(f"Found {len(zip_files)} zip files.")

# 以前のチェックポイントがあればロード（簡易実装）
latest_checkpoint_path = OUTPUT_DIR / 'latest_checkpoint'
if latest_checkpoint_path.exists():
    print(f"Resuming from {latest_checkpoint_path}...")
    # PEFTモデルとしてロードする場合
    model = MusicgenForConditionalGeneration.from_pretrained(
        latest_checkpoint_path,
        quantization_config=quantization_config,
        device_map="auto"
    )
    # LoRAアダプターをロード
    model = get_peft_model(model, lora_config)

# GPU設定 (device_map="auto"を使用しているため、model.to(device)は不要)
device = "cuda" if torch.cuda.is_available() else "cpu"

# CUDAキャッシュをクリアしてメモリを解放
torch.cuda.empty_cache()

# model.to(device) は device_map="auto" を使う場合は不要（または非推奨）
# model.to(device)

for i, zip_file in enumerate(zip_files):
    print(f"\n{'='*40}")
    print(f"Processing Batch {i+1}/{len(zip_files)}: {zip_file.name}")
    print(f"{'='*40}")

    # 1. 解凍
    extract_zip(zip_file, TEMP_DATA_DIR)

    # 2. メタデータ作成
    batch_metadata_path = TEMP_WORK_DIR / 'batch.jsonl'
    success = create_batch_metadata(METADATA_PATH, TEMP_DATA_DIR, batch_metadata_path)

    if not success:
        print("Skipping this batch due to metadata error.")
        continue

    # 3. データセット準備
    dataset = load_dataset("json", data_files=str(batch_metadata_path), split="train")
    dataset = dataset.cast_column("audio", Audio(sampling_rate=processor.feature_extractor.sampling_rate))

    # 前処理の適用
    print("Preprocessing dataset...")
    encoded_dataset = dataset.map(
        lambda x: preprocess_function(x, processor, model), # Pass the model object here
        batched=True,
        remove_columns=dataset.column_names,
        batch_size=4 # メモリに応じて調整
    )

    # 4. トレーニング設定
    # バッチごとにTrainerを作り直すが、modelは同じオブジェクトを使い回すことで学習を継続する
    training_args = TrainingArguments(
        output_dir=str(TEMP_WORK_DIR / "results"),
        per_device_train_batch_size=2, # A100ならもう少し増やせるかも
        gradient_accumulation_steps=4,
        learning_rate=1e-5,
        num_train_epochs=5, # 1バッチあたりのエポック数
        save_steps=1000, # バッチ内での保存頻度（必要なら）
        logging_steps=10,
        fp16=True, # A100/V100ならTrue推奨
        save_total_limit=1,
        remove_unused_columns=False,
        dataloader_num_workers=2,
        report_to="wandb", # WandB有効化
        run_name=f"musicgen-finetuning-batch-{i+1}", # バッチごとにRun名を分ける
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset,
    )

    print("Starting training for this batch...")
    trainer.train()

    # 5. モデル保存
    # バッチ完了ごとにDriveへ保存
    save_path = OUTPUT_DIR / f'checkpoint_batch_{i+1}'
    print(f"Saving model to {save_path}...")
    model.save_pretrained(save_path)
    processor.save_pretrained(save_path)

    # 最新版として上書き
    latest_path = OUTPUT_DIR / 'latest_checkpoint'
    model.save_pretrained(latest_path)
    processor.save_pretrained(latest_path)

    # 6. クリーンアップ
    print("Cleaning up temp data...")
    shutil.rmtree(TEMP_DATA_DIR)
    TEMP_DATA_DIR.mkdir(exist_ok=True)
    # Trainerのクリーンアップ（メモリ解放のため）
    del trainer
    del dataset
    del encoded_dataset
    torch.cuda.empty_cache()

print("All batches processed.")

Loading model: facebook/musicgen-large...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/995 [00:00<?, ?it/s]

MusicgenForConditionalGeneration LOAD REPORT from: facebook/musicgen-large
Key                                           | Status     |  | 
----------------------------------------------+------------+--+-
decoder.model.decoder.embed_positions.weights | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


trainable params: 6,291,456 || all params: 3,429,761,602 || trainable%: 0.1834
Found 139 zip files.

Processing Batch 1/139: archive_batch_0001.zip
Extracting /content/drive/MyDrive/Archive_Wavs/archive_batch_0001.zip to /content/temp_work/data...
Extraction complete.
Creating batch metadata at /content/temp_work/batch.jsonl...
Created metadata with 820 entries.


Generating train split: 0 examples [00:00, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/820 [00:00<?, ? examples/s]

ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'str'>
Returned value: audio_codes
ERROR: model.audio_encoder.encode returned unexpected non-tensor type for audio_codes: <class 'st

ValueError: num_samples should be a positive integer value, but got num_samples=0